In [46]:
import numpy as np
import sklearn
from nltk.corpus import brown
from sklearn.model_selection import train_test_split

class HMM:
    tagged_data = []
    train_corpus = []
    test_corpus = []
    all_tags = []
    all_words = set()
    tags_to_num = {}
    num_to_tags = {}
    transition = {}
    emission = {}
    
    #Preprocess dataset
    def __init__(self):
        tagged_sentences = brown.tagged_sents(tagset='universal')
        for i in range(len(tagged_sentences)):
            sent = []
            sent.append(("^", "^"))
            sent = sent + tagged_sentences[i]
            tagged_data.append(sent)
    
    #Split dataset into train and test data
    def split(self, train_test_ratio = 0.2):
        train_corpus, test_corpus = train_test_split(tagged_data, test_size = 0.2)
    
    #Obtain other parameters like transition and emission probabilities
    def train(self):    
        #All tags in the train corpus
        for sent in train_corpus:
            for (w, t) in sent:
                if t not in all_tags:
                    all_tags.append(t)

        #Count frequency for each tag
        tags_freq = {}
        for t in all_tags:
            tags_freq[t] = 0
        for sent in train_corpus:
            for (w, t) in sent:
                tags_freq[t] += 1

        #tag to number and vice-versa
        for i in range(len(all_tags)):
            tags_to_num[all_tags[i]] = i
            num_to_tags[i] = all_tags[i]

        #All possible words in the set
        for sent in train_corpus:
            for (w, t) in sent:
                all_words.add(w)

        #Count frequency for each (word, tag)
        word_tag_freq = {}
        for w in all_words:
            for t in all_tags:
                word_tag_freq[(w, t)] = 0
        for sent in train_corpus:
            for (w, t) in sent:
                word_tag_freq[(w, t)] += 1

        #evaluate transition  counts
        for t1 in all_tags:
            for t2 in all_tags:
                transition[(t1, t2)] = 0
        for sent in train_corpus:
            for i in range(len(sent)-1):
                transition[(sent[i][1], sent[i+1][1])] += 1

        #evaluate transition and emission probabilities
        for (t1, t2) in transition.keys():
            transition[(t1, t2)] /= tags_freq[t1]
        for (w, t) in word_tag_freq.keys():
            emission[(w, t)] = word_tag_freq[(w, t)]/tags_freq[t]
    
    #Implements Viterbi Algorithm
    def viterbi(sentence):
        #returns predicted tag sequence
        len_sent = len(sentence)
        len_tagset = len(all_tags)

        #SEQSCORE and BACKPTR arrays
        SEQSCORE = [[0 for i in range(len_sent)] for j in range(len_tagset)]
        BACKPTR =  [[0 for i in range(len_sent)] for j in range(len_tagset)]

        null_tag = tags_to_num["^"]
        #initialise the null tag
        SEQSCORE[null_tag][0] = 1

        for i in range(1, len_sent):#Corresponds to a given word sentence[i]
            for cidx, ctag in enumerate(all_tags):#Ending at current tag all_tags[j]

                optimal_prob = 0 #Includes transitional probabilites
                optimal_tag = 0

                for pidx, ptag in enumerate(all_tags):#Previous tag all_tags[k]
                    prob_k_j_i = SEQSCORE[pidx][i-1]*transition[(ptag, ctag)]
                    if prob_k_j_i > optimal_prob:
                        optimal_prob = prob_k_j_i
                        optimal_tag = pidx

                if sentence[i] in all_words:
                    SEQSCORE[cidx][i] = optimal_prob*emission[(sentence[i], ctag)]
                else:
                    SEQSCORE[cidx][i] = optimal_prob
                BACKPTR[cidx][i] = optimal_tag

        #Sequence identification step
        CT = 0
        optimal_prob = 0
        for i in range(len_tagset):
            if SEQSCORE[i][len_sent-1]>optimal_prob:
                optimal_prob = SEQSCORE[i][len_sent-1]
                CT = i

        pred_tags = [CT for i in range(len_sent)]
        for i in reversed(range(len_sent-1)):
            pred_tags[i] = BACKPTR[pred_tags[i+1]][i+1]

        pred_tags = [num_to_tags[idx] for idx in pred_tags]
        return pred_tags
    
    def get_accuracy(self):
        count_accurate = 0
        count_total = 0
        for sentence in test_corpus:
            untagged_sentence = []
            for (w, t) in sentence:
                untagged_sentence.append(w)
            predicted_tags = viterbi(untagged_sentence, transition, emission, all_words, all_tags, tags_to_num, num_to_tags)
            for i in range(len(sentence)):
                if predicted_tags[i] == sentence[i][1]:
                    count_accurate += 1
                count_total += 1
        return count_accurate/count_total

hmm = HMM()
hmm.split()
hmm.train()
print(hmm.get_accuracy())

0.9595177670194431
